In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('once')
from sklearn.tree import DecisionTreeRegressor
import datetime as dt

In [64]:
df = pd.read_csv('../datos/casual/15-casual-cod_est.csv', index_col=0)
df.head(2)

,atemp,hum,windspeed,dteday,season,yr,holiday,weekday,weathersit,casual,...,mnth_3,mnth_4,mnth_5,mnth_6,mnth_7,mnth_8,mnth_9,mnth_10,mnth_11,mnth_12
0,-0.456399,0.855582,-0.208909,2018-01-01,0,0,0,3,1,331,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.492863,0.332722,0.687550,2018-01-02,0,0,2,1,1,131,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
df.dtypes

atemp         float64
hum           float64
windspeed     float64
dteday         object
season          int64
yr              int64
holiday         int64
weekday         int64
weathersit      int64
casual          int64
mnth_1        float64
mnth_2        float64
mnth_3        float64
mnth_4        float64
mnth_5        float64
mnth_6        float64
mnth_7        float64
mnth_8        float64
mnth_9        float64
mnth_10       float64
mnth_11       float64
mnth_12       float64
dtype: object

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 729
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   atemp       730 non-null    float64
 1   hum         730 non-null    float64
 2   windspeed   730 non-null    float64
 3   dteday      730 non-null    object 
 4   season      730 non-null    int64  
 5   yr          730 non-null    int64  
 6   holiday     730 non-null    int64  
 7   weekday     730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   casual      730 non-null    int64  
 10  mnth_1      730 non-null    float64
 11  mnth_2      730 non-null    float64
 12  mnth_3      730 non-null    float64
 13  mnth_4      730 non-null    float64
 14  mnth_5      730 non-null    float64
 15  mnth_6      730 non-null    float64
 16  mnth_7      730 non-null    float64
 17  mnth_8      730 non-null    float64
 18  mnth_9      730 non-null    float64
 19  mnth_10     730 non-null    f

In [67]:
df.head(2)

,atemp,hum,windspeed,dteday,season,yr,holiday,weekday,weathersit,casual,...,mnth_3,mnth_4,mnth_5,mnth_6,mnth_7,mnth_8,mnth_9,mnth_10,mnth_11,mnth_12
0,-0.456399,0.855582,-0.208909,2018-01-01,0,0,0,3,1,331,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.492863,0.332722,0.687550,2018-01-02,0,0,2,1,1,131,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
df.drop(['dteday'], axis = 1, inplace=True)

---

In [69]:
X = df.drop("casual", axis = 1)
y = df["casual"]

In [70]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42) 

In [71]:
arbol = DecisionTreeRegressor(random_state =0)
arbol.fit(x_train, y_train)

DecisionTreeRegressor(random_state=0)

In [72]:
#fig = plt.figure(figsize = (10,6))
#tree.plot_tree(arbol, feature_names = x_train.columns, filled = True)
#plt.show()

In [73]:
max_features = np.sqrt(len(x_train.columns))
max_features

4.47213595499958

In [74]:
print(arbol.tree_.max_depth)

23


In [75]:
param = {"max_depth": [4,6,8,10,12,14,16],
        "max_features": [1,2,3,4],
        "min_samples_split": [10,40,70],
        "min_samples_leaf": [10,40,70]} 

In [76]:
gs = GridSearchCV(
            estimator=DecisionTreeRegressor(), 
            param_grid= param, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error")
gs.fit(x_train, y_train)



GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [4, 6, 8, 10, 12, 14, 16],
                         'max_features': [1, 2, 3, 4],
                         'min_samples_leaf': [10, 40, 70],
                         'min_samples_split': [10, 40, 70]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [77]:
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=12, max_features=4, min_samples_leaf=10,
                      min_samples_split=10)

In [78]:
#fig = plt.figure(figsize=(40, 20))
#tree.plot_tree(mejor_modelo, feature_names=x_train.columns, filled=True);

In [79]:
y_pred_test_dt2 = mejor_modelo.predict(x_test)
y_pred_train_dt2 = mejor_modelo.predict(x_train)

In [80]:

def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [81]:
dt_results1 = metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, "1_Dtree_casual_est-cod")
dt_results1

,MAE,MSE,RMSE,R2,set,modelo
0,402.552731,365778.849167,604.796535,0.238633,test,1_Dtree_casual_est-cod
1,337.584111,238440.750812,488.303953,0.490191,train,1_Dtree_casual_est-cod


Random forest

In [93]:
X = df.drop("casual", axis = 1)
y = df["casual"]

In [94]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [95]:
param = {"max_depth": [6,8,10,12],
        "max_features": [1,2,3,4],
        "min_samples_split": [10,40,70],
        "min_samples_leaf": [10,40,70]} 

In [96]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(), 
            param_grid= param, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 


In [97]:
gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [6, 8, 10, 12],
                         'max_features': [1, 2, 3, 4],
                         'min_samples_leaf': [10, 40, 70],
                         'min_samples_split': [10, 40, 70]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [98]:
bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=12, max_features=4, min_samples_leaf=10,
                      min_samples_split=10)

In [99]:
'''for arbol in tqdm(bosque.estimators_):
    fig = plt.figure(figsize=(40, 20))
    tree.plot_tree(arbol, feature_names= x_train.columns, filled = True)
    plt.show()'''

'for arbol in tqdm(bosque.estimators_):\n    fig = plt.figure(figsize=(40, 20))\n    tree.plot_tree(arbol, feature_names= x_train.columns, filled = True)\n    plt.show()'

In [100]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
   
   
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df


In [101]:
dt_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "2_RForest_casual_cod_estand")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,285.600487,153412.860884,391.679539,0.680672,test,2_RForest_casual_cod_estand
1,279.212318,148395.024559,385.220748,0.682717,train,2_RForest_casual_cod_estand


In [102]:
df_todos_resultados = pd.concat([dt_results, dt_results1], axis = 0)
df_todos_resultados


,MAE,MSE,RMSE,R2,set,modelo
0,285.600487,153412.860884,391.679539,0.680672,test,2_RForest_casual_cod_estand
1,279.212318,148395.024559,385.220748,0.682717,train,2_RForest_casual_cod_estand
0,402.552731,365778.849167,604.796535,0.238633,test,1_Dtree_casual_est-cod
1,337.584111,238440.750812,488.303953,0.490191,train,1_Dtree_casual_est-cod


In [103]:
df_todos_resultados.to_csv('../datos/casual/16-casual-metricas-estand.csv')

# Random Forest es la mejor opcion